# invoiceDemoLibrary

# Start

In [1]:
def calc_keys(assets):
    _keys = []
    for asset in assets:
        _keys.append(asset['Key'])
    return _keys    

def getUsedRoles(roles):
    _all=[]
    for element in roles:
        for role in element['roles']:
            if role not in _all:
                _all.append(role)
    return _all

In [2]:
from ipynb.fs.full.facturxWidget import *
#from ipynb.fs.full.kubernetesTool import *
#from ipynb.fs.full.invoiceSmartcontract import *

from ipynb.fs.full.blockchain import *

In [3]:
_blockchain = Blockchain(channel_name='invoicing26',shared_fs='10.15.130.111:80',sharepoint=SharePoint())
cc = Chaincode()
_blockchain.addPeer('aubonmoulin', hostname='fr',ca_country='FR',ca_province='Provence-Alpes-Cote d\'Azur',ca_locality='Malaucene')
_blockchain.addPeer('lyon-impots', hostname='gouv.fr',ca_country='FR',ca_province='Auvergne-Rhone-Alpes',ca_locality='Lyon')
_blockchain.addPeer('austriafactor', hostname='at',ca_country='AT',ca_province='Steiermark',ca_locality='Graz')
_blockchain.addPeer('megustaolive', hostname='es',ca_country='ES',ca_province='Andalusia',ca_locality='Dos Hermanas')

_blockchain.installWalletTool()

In [4]:
_user = User(peer=_blockchain.peers['aubonmoulin'], name='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name)

In [5]:
roles=_user.smart_contract.getAllRoles()[0]
roleTransactions=_user.smart_contract.getRoleTransactions()[0]
usedRoles=getUsedRoles(roles)

In [6]:
#User(peer=_blockchain.peers['aubonmoulin'], name='Stephanie.Hoarau@hotelsaintdenis.re', channel_name=_blockchain.channel_name).smart_contract.listInvoices()
assets = _user.smart_contract.listInvoices()[0]
_keys = calc_keys(assets)

In [7]:
invoices = get_invoices()
_invoice = invoices.iloc[3]

# Demo Functions

In [ ]:
def get_current_user_with_role(rolename,asset):    
    if rolename == 'Buyer':
        _cn = get_cert_cn(asset['buyer'])
        return _blockchain.getUser(_cn)
    elif  rolename == 'Seller':  
        _cn = get_cert_cn(asset['owner'])
        return _blockchain.getUser(_cn)
    elif  rolename == 'Factor':   
        _cn = get_cert_cn(asset['owner'])
        return _blockchain.getUser(_cn)
    elif  rolename == 'TaxInspector':    
        _cn = get_cert_cn(asset['owner'])
        _owner = _blockchain.getUser(_cn)                    
        _cn = get_cert_cn(_owner.smart_contract.getRoles()[0]['taxInspector'])
        return _blockchain.getUser(_cn)    

### Widgets

In [ ]:
class BlockchainExplorerWidget(object):
    def __init__(self):
        # explore blocks
        self.block_btn = widgets.Button(description=f'Explore Last Block', disabled=False,button_style='success', icon='check')    
        self.block_btn.on_click(self.block_btn_reload_clicked)
        self.block_text = widgets.Text(description = 'height', style=style, layout=layout)
        
        self.block_text.on_submit(self.on_value_submit_block_text)   
    
        self.block_out = widgets.Output()
        self.block_explorer = widgets.VBox([self.block_btn,self.block_text,self.block_out])                
        
        box_layout2 = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='flex-end',
                width='34%')
        self.block_explorer.layout=box_layout2 
        
        
    def on_value_submit_block_text(self,change): 
        self.update_block_explorer(change.value)

        
    def _display_block_ui(self,_block,id):
        html = widgets.HTML(
                value='Transaktion',
                description=' ',
                style= {'description_width': '250px'}, layout =  widgets.Layout(margin='0px 0 0 0px',width= '250px')
                )
        display(html)
        _t = Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).decode_block_transaction(block = _block)
        facturx_widgets(_t,'/', hideUnused=True, hide=False, style = {'description_width': '250px'}, layout =  widgets.Layout(margin='0 0 0 250px',width= '250px')).output
        _b = Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).decode_block_data(block = _block)
        _w= facturx_widgets(_b,'/', hideUnused=True, hide=False)  
        _w.container.fields[' '].widget.value= 'Block ' + str(id)
        
    def _display_block_json1(self,_block,id):
        style= {'description_width': '250px'}
        layout =  widgets.Layout(margin='20px 0 0 0px',width= '500px')
        #style= {'description_width': '0px'}, 
        #layout =  widgets.Layout(margin='0px 0px 0px 0px',width= '250px')
        html = widgets.HTML(
                value="<b>" + 'Transaktion' +  "</b>",
                description=' ',
                style=style, layout=layout
                )
        display(html)
        
        _t = Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).decode_block_transaction(block = _block)
        html = widgets.HTML(
            value=str(_t),
            description=' ',
            style=style, layout=layout
        )
        display(html)

        
        _b = Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).decode_block_data(block = _block)
        html = widgets.HTML(
                value="<b>" + 'Block ' + str(id) +  "</b>",
                description=' ',
                style=style, layout=layout
                )
        display(html)
        html = widgets.HTML(
        value=str(_b),
        description=' ',
        style=style, layout=layout
        )
        display(html)
                
        
    def changed(self,b):
        id = 1
        Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).uploadBlockToShare(id=_id)
        _block=Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).getBlockFromShare(id=_id)
        with self.block_out:  
            clear_output() 
            if b['name'] == 'value':
                if text.value:
                    self._display_block_json1(_block,id)
                else:
                    self._display_block_ui(_block,id)        
        
    def update_block_explorer(self,id):
        _id = int(id) - 1
        Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).uploadBlockToShare(id=_id)
        _block=Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).getBlockFromShare(id=_id)
        with self.block_out:  
            clear_output()  
            #text = widgets.Checkbox(description = 'json',value = False, style= {'description_width': '0px'}, layout =  widgets.Layout(margin='0px 0px 0px 0px',width= '250px'))
            text = widgets.Checkbox(description = 'json',value = False, style= {'description_width': '250px'}, layout =  widgets.Layout(margin='20px 0 0 0px',width= '500px'))
            display(text) 
            #text.observe(changed)
            self._display_block_ui(_block,id)
            #self._display_block_json1(_block,id)
        
  # !!! depends on peer, channel     
    def block_btn_reload_clicked(self,i):   
        height=Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).get_height()
        self.block_text.value = str(height)
        self.update_block_explorer(height)        
        

## InvoiceWidget

In [ ]:
import functools

class InvoiceWidget(object):
    def __init__(self,user=None,usedRoles=None,asset_keys=None):
        
        self.roleOptions = widgets.Dropdown(options=usedRoles)
        self.roleOptions.observe(self.reload_clicked, names='value')

        self.btn = widgets.Button(description=f'From Blockchain', disabled=False,button_style='success', icon='check')    
        self.btn.on_click(self.reload_clicked)

        self.assetOptions = widgets.Dropdown(options=asset_keys)
        self.assetOptions.observe(self.asset_selected, names='value')
        self.options = widgets.VBox([self.assetOptions, self.roleOptions])

        self.assetOptions.value='asset14'    
        self.user  = user
        self.asset = self.user.smart_contract.listInvoice(self.assetOptions.value)[0]
        
        self.widget = facturx_widgets(self.asset,'/', hideUnused=False, hide=True) 
        self.widget_vbox = widgets.VBox([self.btn,self.widget.output])
        
        self.block_explorer = BlockchainExplorerWidget()
        
        self.HBox   = widgets.HBox([self.widget_vbox, self.options,self.block_explorer.block_explorer]) 
        self.enable_fields_for_role()
        
        # fix empty header line
        self.widget.container.fields[' '].widget.layout=widgets.Layout(display='flex',height='5px')
        box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='flex-end',
                width='24%')
        self.widget_vbox.layout=box_layout 
        
        #display(self.HBox)
        
        #self.reload_clicked('')
        
    def widget_load_asset(self,asset):
        self.widget.output.clear_output()
        self.widget.container = FieldContainer()
        with self.widget.output:
            self.widget.addChilds(asset,'/', hideUnused=True)

    def reload_clicked(self,i):
        id=self.assetOptions.value
        #print(self.asset)
        self.user  = get_current_user_with_role(self.roleOptions.value, self.asset)
        self.asset = self.user.smart_contract.listInvoice(id)[0]
        self.widget_load_asset(self.asset)
        self.init_fields_with_role(self.user,id) 
        
        # fix empty header line
        self.widget.container.fields[' '].widget.layout=widgets.Layout(display='flex',height='5px')

    def role_selected(self,x): 
        self.user = get_current_user_with_role(roleOptions.value,asset)
        self.init_fields_with_role(self.user) 
        
    def init_fields_with_role(self,user,id):   
        if self.roleOptions.value == 'Buyer':
            self.init_fields_buyer(user=user,id=id)
        elif self.roleOptions.value == 'Seller':
            self.init_fields_seller(user=user,id=id)
        elif self.roleOptions.value == 'TaxInspector':    
            self.init_fields_taxinspector(user=user,id=id)
        elif self.roleOptions.value == 'Factor':    
            self.init_fields_factor(user=user,id=id)

    def asset_selected(self,x): 
        self.reload_clicked('')

      
    def disable_all_fields(self):
        for key,value in self.widget.container.fields.items():
            value.widget.disabled=True

    def enable_fields(self,fieldnames):
        for fieldname in fieldnames:
            self.widget.container.fields[fieldname].widget.disabled=False
            
    def enable_fields_for_role(self):
        rolename=self.roleOptions.value
        self.disable_all_fields()
        if rolename == 'Buyer':
            self.enable_fields(['received','receivedOrder'])
        elif  rolename == 'Seller':   
            self.enable_fields(['sold','claimPaid','claimPaidBy','owner'])
        elif  rolename == 'Factor':   
            self.enable_fields(['claimPaid','claimPaidBy'])
        elif  rolename == 'TaxInspector':   
            self.enable_fields(['taxReceived'])    
            
            
    def init_fields_buyer(self,user=None,id=None):
    
        def received_changed(b,user=None,id=None):
            if b['name'] == 'value':
                user.smart_contract.receivedInvoice(id=id)[1]

        def receivedOrder_changed(b,user=None,id=None):
            if b['name'] == 'value':
                user.smart_contract.receivedOrder(id=id)[1]    

        self.widget.container.fields['received'].widget.observe(functools.partial(received_changed,user=user,id=id))
        self.widget.container.fields['receivedOrder'].widget.observe(functools.partial(receivedOrder_changed,user=user,id=id))
        self.enable_fields_for_role()


    def init_fields_seller(self,user=None,id=None):

        def sold_changed(b,user=None,id=None):
            if b['name'] == 'value':
                name=self.widget.container.fields['owner'].widget.value
                # FIX problem for factor
                self.asset['owner'] = name
                user.smart_contract.transferInvoice(id=id,name=name)[1]

        def claimPaid_changed(b,user=None,id=None):
            if b['name'] == 'value':
                user.smart_contract.receivedPayment(id=id, payer= self.widget.container.fields['claimPaidBy'].widget.value)

        self.enable_fields_for_role()    
        self.widget.container.fields['sold'].widget.observe(functools.partial(sold_changed,user=user,id=id))
        self.widget.container.fields['claimPaid'].widget.observe(functools.partial(claimPaid_changed,user=user,id=id))  
        self.replace_owner_field_by_dropdown()


    def init_fields_factor(self,user=None,id=None):

        def claimPaid_changed(b,user=None,id=None):
            if b['name'] == 'value':
                user.smart_contract.receivedPayment(id=id, payer= self.widget.container.fields['claimPaidBy'].widget.value)

        self.enable_fields_for_role()    
        self.widget.container.fields['claimPaid'].widget.observe(functools.partial(claimPaid_changed,user=user,id=id))  


    def init_fields_taxinspector(self,user=None,id=id):
        def taxReceived_changed(b,user=None,id=None):
            if b['name'] == 'value':
                 user.smart_contract.taxReceived(id=id)  

        self.widget.container.fields['taxReceived'].widget.observe(functools.partial(taxReceived_changed,user=user,id=id))
        self.enable_fields_for_role()    

    # !!! roles
    def create_dropdown_owner(self):
        _certs=[self.widget.container.fields['owner'].widget.value]
        _certs = _certs + get_person_certs_with_role('Factor',roles)
        return widgets.Dropdown(options=_certs, value = _certs[0], description="owner", style=style, layout=layout)

    def replace_owner_field_by_dropdown(self):
        _owners = self.create_dropdown_owner()
        self.widget.container.fields['owner'] = Field(widget=_owners,path=self.widget.container.fields['owner'].path)
        self.widget.redisplay()    

In [ ]:
def role_selected(x): 
    if roleOptions.value == 'Buyer':
        init_fields_buyer(user=buyer,peer=peer_buyer)
    elif roleOptions.value == 'Seller':
        init_fields_seller(user=owner,peer=peer_owner)
    elif roleOptions.value == 'TaxInspector':    
        init_fields_taxinspector(user=taxinspector,peer=peer_taxinspector)
    elif roleOptions.value == 'Factor':    
        init_fields_factor(user=factor,peer=peer_factor)

In [ ]:
def create_address_widget(_invoice):
    _widget1 =  widgets.HBox([widgets.VBox([facturx_widgets(_invoice,'/exchangedDocumentContext', hideUnused=True, hide=True).output ,
                              facturx_widgets(_invoice,'/exchangedDocument', hideUnused=True, hide=True).output]) ,
                              facturx_widgets(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty', hideUnused=True, hide=True).output ,
                              facturx_widgets(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty', hideUnused=True, hide=True).output])
    #display (_widget1)
    return _widget1


def create_pod_widget(_invoice,blockchain):
    pod_list = []
    for key,peer in blockchain.peers.items():
        pod_list.append(facturx_widgets(peer.__dict__,'/', hideUnused=True, hide=True).output)
    _widget = widgets.VBox(pod_list)
    return _widget

def create_cluster_widget():
    output = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'get','service','hl-fabric-cc-external-invoice-net','-o','json'])
    _chaincode_service = json.loads(output.decode('ascii'))
    _chaincode_service_fields = {'name':_chaincode_service['metadata']['name'], 'nodePort':_chaincode_service['spec']['ports'][0]['nodePort']}

    _chaincode_widget = facturx_widgets(hideUnused=True, hide=True)
    #_header = _chaincode_widget.addHeader(text='Chaincode',bold=True, path='',hide=True )
    #_chaincode_widget.container.add(widget=_header)
    _chaincode_widget.append(data=_chaincode_service_fields)
    
    shared_fs_host=cc.shared_fs
    _sharepoint_widget = facturx_widgets({'data share':shared_fs_host},'/', hideUnused=True, hide=True).output

    _peer_pods = create_pod_widget(_invoice,_blockchain)
    _orderer = ordererpeer(name='orderer-org1-scray-org',prefix='orderer').__dict__
    _orderer_pod = facturx_widgets(_orderer,'/', hideUnused=True, hide=True).output

    _column2 = widgets.VBox([_orderer_pod, _chaincode_widget.output, _sharepoint_widget])
    #_chaincode_widget.redisplay()
    return widgets.HBox([_peer_pods, _column2])




In [ ]:
def get_current_user_with_role_1(rolename,asset):    
    if rolename == 'Buyer':
        _cn = get_cert_cn(asset['buyer'])
        return _blockchain.getUser(_cn)
    elif  rolename == 'Seller':  
        _cn = get_cert_cn(asset['owner'])
        return _blockchain.getUser(_cn)
    elif  rolename == 'Factor':   
        _cn = get_cert_cn(asset['owner'])
        return _blockchain.getUser(_cn)
    elif  rolename == 'TaxInspector':    
        _cn = get_cert_cn(asset['owner'])
        _owner = _blockchain.getUser(_cn)                    
        _cn = get_cert_cn(_owner.smart_contract.getRoles()[0]['taxInspector'])
        return _blockchain.getUser(_cn)

## role

In [ ]:
def get_CN_of_invoice(invoice,party):
    _name=strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/' + party + '/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))
    _email=getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/' + party + '/definedTradeContact/')[0]['emailURIUniversalCommunication']['uriid']['value']
    return _name + '@' + _email.split('@')[1]
seller = getUser(_blockchain,get_CN_of_invoice(_invoice,'sellerTradeParty'))
buyer  = getUser(_blockchain,get_CN_of_invoice(_invoice,'buyerTradeParty'))   

In [ ]:
#_roles = seller.smart_contract.getRoles()[0]
#display_role(roles, role_transctions, role,user,peer)
#_role='Seller'
#_role_transactions = get_transactions_by_role(_role)

In [ ]:
def display_role(roles, role_transctions, role,user,peer):
    _role_widget = facturx_widgets( hideUnused=True, hide=True, layout = {'width': '390px'}, style = {'description_width': '110px'})
    _header = _role_widget.addHeader(text='Rolle',bold=True, path='',hide=True )
    _role_widget.container.add(widget=_header)
    _role_widget.append(data=roles) 
    _header = _role_widget.addHeader(text='User',bold=True, path='',hide=True )
    _role_widget.container.add(widget=_header)
    _role_widget.append(data={'user':user,'peer':peer.name})
    
    _role_transctions_widget = facturx_widgets( hideUnused=True, hide=True, layout = {'width': '300px'}, style = {'description_width': '50px'})
    _header = _role_transctions_widget.addHeader(text='Transaktionen: ' + role,bold=True, path='',hide=True  )
    _role_transctions_widget.container.add(widget=_header)
    _role_transctions_widget.append(data=role_transctions)

    block_explorer = BlockchainExplorerWidget()
    _hbox = widgets.HBox([_role_widget.output,_role_transctions_widget.output, block_explorer.block_explorer])
    display(_hbox)
    _role_widget.redisplay()
    _role_transctions_widget.redisplay()

def get_transactions_by_role(role):
    for _role in roleTransactions:
        if _role['role'] == role:
            return _role['transactions']

        
def update_role_widget(user=None,peer=None,role=None,roles=None):
    #_roles = getRoles(peer, user=user)[0]
    _roles = roles
    _role_transactions = get_transactions_by_role(role)
    
    if isinstance(_roles, str):
        _roles=''
    
    with roleWidget_out:
        clear_output()
        display_role(_roles, _role_transactions,role,user,peer)

def role_widget_selected(x): 
    if roleWidgetOptions.value == 'Buyer':
        buyer=get_CN_of_invoice(_invoice,'buyerTradeParty')
        user = getUser(_blockchain,buyer)
        roles = user.smart_contract.getRoles()[0]
        update_role_widget(user=buyer,peer=user.peer,role=roleWidgetOptions.value,roles=roles)
    elif roleWidgetOptions.value == 'Seller':
        owner= get_CN_of_invoice(_invoice,'sellerTradeParty')
        seller = getUser(_blockchain,owner)
        roles = seller.smart_contract.getRoles()[0]
        update_role_widget(user=owner,peer=seller.peer,role=roleWidgetOptions.value,roles=roles)
    elif roleWidgetOptions.value == 'TaxInspector':   
        pass
        #update_role_widget(user=taxinspector,peer=peer_taxinspector,role=roleWidgetOptions.value)
    elif roleWidgetOptions.value == 'Factor':    
        pass
        #update_role_widget(user=factor,peer=peer_factor,role=roleWidgetOptions.value)                     
        

roleWidgetOptions = widgets.Dropdown(options=usedRoles)
roleWidgetOptions.observe(role_widget_selected, names='value')     
roleWidget_out = widgets.Output()

In [ ]:
#get_current_user_with_role(_widget.roleOptions.value, _widget.asset)
_widget.roleOptions.value
#_widget.user.__dict__
_widget.asset

# Invoice Widget

In [ ]:
cluster_widget = create_cluster_widget()

facturx__widget = facturx_widgets(data=_invoice,path='/supplyChainTradeTransaction', hideUnused=True, hide=True).output
address_widget = create_address_widget(_invoice)
roles_widget = widgets.HBox([roleWidgetOptions,roleWidget_out])
_widget=InvoiceWidget(user=_user,usedRoles=usedRoles,asset_keys=_keys)  

tab  = widgets.Tab(children = [cluster_widget, facturx__widget, address_widget,roles_widget,_widget.HBox])
titles = ['cluster','Factur-X Rechnung','Adressen', 'Rollen', 'Invoice Widget']

[tab.set_title(i, title) for i, title in enumerate(titles)]
display(tab)

In [ ]:
facturx_widgets(self.asset,'/', hideUnused=False, hide=True) 

In [ ]:
def myfunc(**kwargs):
        # kwargs is a dictionary.
        for k,v in kwargs.items():
             print ("%s = %s" % (k, v))
        #if 'image' in kwargs.keys():
        #    print('found')  
        _widget = facturx_widgets(kwargs,'/', hideUnused=False, hide=True) 
        return _widget
        
_widget1 = myfunc(name='aubonmoulin', hostname='fr',ca_country='FR',ca_province='Provence-Alpes-Cote d\'Azur',ca_locality='Malaucene')     

In [ ]:
display(_widget1.output)

In [ ]:
#facturx_widgets(_blockchain.peers['aubonmoulin'].organizations,'/', hideUnused=False, hide=True)
#_blockchain.peers['aubonmoulin'].organizations['aubonmoulin.fr'].dict()
_obj = _blockchain.peers['lyon-impots'].organizations['lyon-impots.gouv.fr'].users['Auguste.Boucher@lyon-impots.gouv.fr'].dict()['parameters']
_obj = _blockchain.peers['lyon-impots'].organizations['lyon-impots.gouv.fr'].dict()
_obj = _blockchain.peers['aubonmoulin'].dict()['parameters']
facturx_widgets(_obj,'/', hideUnused=False, hide=True).output 

In [ ]:
for key,_peer in _blockchain.peers.items():
    print(_peer.dict()['parameters'])
    for key,_organization in _peer.organizations.items():
        print(_organization.dict()['parameters'])

In [ ]:
_blockchain.__dict__

In [ ]:
_blockchain.peers['lyon-impots'].organizations['lyon-impots.gouv.fr'].users['Auguste.Boucher@lyon-impots.gouv.fr'].dict()['parameters']

In [ ]:
Organization().__dict__